In [44]:
#Imports
import datetime
from selenium import webdriver
from datetime import timedelta
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By




In [27]:
 class SystemDataSource:
        
    source = ""
    initialDate = ""
    useIframe = False
    iframeId = ""
    initialDateHtmlElementId = ""
    finalDateHtmlElementId = ""

    def __init__(self):
        self.data = []

In [88]:
#Instacia um objeto datasource com as informações da prefeitura de Uberaba-MG
dataSource = SystemDataSource()
dataSource.source = "http://www.uberaba.mg.gov.br/portal/conteudo,13641"
#dataSource.initialDate = datetime.date(2010,5, 27)
dataSource.initialDate = datetime.date(2017,11, 23)
dataSource.useIframe = True
dataSource.iframeId = "iframeContabil"
dataSource.initialDateHtmlElementId = "dtInicial"
dataSource.finalDateHtmlElementId = "dtFinal"

DATE_PATTERN = "%d/%m/%Y"

In [89]:
#Instacia o webdriver do navegador Chrome
navigator = webdriver.Chrome()

#Navega para a página da prefeitura da cidade
navigator.get(dataSource.source)

#Verifica se a página utiliza iframe
if(dataSource.useIframe):
    navigator.switch_to_frame(navigator.find_element_by_id(dataSource.iframeId))

#Instancia um webDriverWait para podermos fazer com que o navegador espere algum evento externo    
wait = WebDriverWait(navigator, 10)    

dateTimeLoop = dataSource.initialDate
dateTimeNow = datetime.datetime.now().date()

while(dateTimeLoop <= dateTimeNow):
    
    initialDateValue = dateTimeLoop.strftime(DATE_PATTERN)
    dateTimeLoop = dateTimeLoop + timedelta(days=10)
    finalDateValue = dateTimeLoop.strftime(DATE_PATTERN)
    dateTimeLoop = dateTimeLoop + timedelta(days=1)


    #Altera a data inicial da busca dos filtros
    initialDateObject  = navigator.find_element_by_id(dataSource.initialDateHtmlElementId)
    initialDateObject.click()
    initialDateObject.send_keys(initialDateValue)
    wait.until(EC.text_to_be_present_in_element_value((By.ID, dataSource.initialDateHtmlElementId),initialDateValue))

    #Altera a data final da busca dos filtros
    finalDateObject = navigator.find_element_by_id(dataSource.finalDateHtmlElementId)
    finalDateObject.click()
    finalDateObject.send_keys(finalDateValue)
    wait.until(EC.text_to_be_present_in_element_value((By.ID, dataSource.finalDateHtmlElementId),finalDateValue))

    #Clica no botão para fazer a consulta
    navigator.find_element_by_id("btnEnviar").click()

    #Aguarda até o retorno do AJAX
    wait.until(EC.visibility_of_element_located((By.ID, "divResult1")))

    #Clica no botão para fazer o download
    navigator.find_element_by_class_name("buttons-csv").click()

navigator.switch_to_default_content()

#navigator.close()

